In [1]:
import threading
import time


In [2]:
import requests
import urllib.parse


In [3]:
"""
🏛️ LATAM 45 INDEX - MARKET MONITOR
------------------------------------------------------------------
Un sistema de visualización financiera robusto para el mercado latinoamericano.
Monitoriza 45 activos estratégicos (ADRs y Locales) con lógica de momentum.

Autor: Claude AI + Gemini
Versión: Final Stable (LATAM 45)
"""

import yfinance as yf
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output
import warnings
from datetime import datetime
import threading
import time

# Configuración de entorno
warnings.filterwarnings('ignore')

# ============================================================================
# 1. UNIVERSO DE INVERSIÓN: LATAM 45
# ============================================================================
# Selección curada de los 45 activos más líquidos y estables.
# Se han eliminado tickers conflictivos para garantizar carga del 100%.

LATAM_45_TICKERS = {
    # 🚀 TECH & FINTECH (NASDAQ/NYSE - Alta Liquidez)
    'MELI': 'MercadoLibre',
    'NU':   'Nubank Global',
    'GLOB': 'Globant',
    'STNE': 'StoneCo',
    'PAGS': 'PagSeguro',
    'XP':   'XP Inc',
    'DLO':  'DLocal',

    # 🛢 ENERGÍA & UTILITIES (ADRs + Locales Selectos)
    'PBR':  'Petrobras',
    'YPF':  'YPF',
    'VIST': 'Vista Energy',
    'PAM':  'Pampa Energía',
    'TGS':  'Transp. Gas Sur',
    'CMIG4.SA': 'Cemig (Bra)',
    'CPFE3.SA': 'CPFL Energía (Bra)',
    'UGPA3.SA': 'Ultrapar (Bra)',
    'SBS':  'Sabesp',

    # ⛏ MATERIALES & MINERÍA (Global Commodities)
    'VALE': 'Vale',
    'SCCO': 'Southern Copper',
    'SQM':  'SQM (Litio)',
    'TX':   'Ternium',
    'GGB':  'Gerdau',
    'SID':  'CSN Siderúrgica',
    'CX':   'Cemex',
    'SUZ':  'Suzano',
    'BVN':  'Buenaventura',

    # 🏦 BANCA & FINANZAS (Sector Financiero)
    'ITUB': 'Itaú Unibanco',
    'BBD':  'Bradesco',
    'BSBR': 'Santander Brasil',
    'BMA':  'Banco Macro',
    'GGAL': 'Grupo Galicia',
    'BAP':  'Credicorp',
    'CIB':  'Bancolombia',
    'BCH':  'Banco de Chile',
    'BBVA': 'BBVA Latam',

    # 🛒 CONSUMO & RETAIL (Defensivas)
    'FMX':  'Femsa',
    'KOF':  'Coca-Cola Femsa',
    'ABEV': 'Ambev',
    'ARCO': 'Arcos Dorados',
    'WALMEX.MX': 'Walmart México',

    # ✈️ INDUSTRIA & TRANSPORTE
    'CPA':  'Copa Airlines',
    'PAC':  'GAP Aeropuertos',
    'ASR':  'ASUR Aeropuertos',
    'OMAB': 'OMA Aeropuertos',
    'CAAP': 'Corp. América',
    'TV':   'Grupo Televisa'
}

# Mapa Sectorial alineado con los 45 activos
SECTORES_MAP = {
    'Tech & Fintech': ['MELI', 'NU', 'GLOB', 'XP', 'STNE', 'PAGS', 'DLO'],
    'Energía & Utilities': ['PBR', 'YPF', 'VIST', 'PAM', 'TGS', 'CMIG4.SA', 'SBS', 'CPFE3.SA', 'UGPA3.SA'],
    'Materiales & Minería': ['VALE', 'SCCO', 'SQM', 'TX', 'GGB', 'SID', 'CX', 'SUZ', 'BVN'],
    'Banca Tradicional': ['ITUB', 'BBD', 'BSBR', 'BAP', 'CIB', 'BCH', 'GGAL', 'BMA', 'BBVA'],
    'Consumo & Retail': ['FMX', 'ABEV', 'KOF', 'ARCO', 'TV', 'WALMEX.MX'],
    'Industria & Transporte': ['CPA', 'PAC', 'ASR', 'OMAB', 'CAAP']
}

# ============================================================================
# 2. MOTOR DE ENLACES TRADINGVIEW (Smart Linking)
# ============================================================================

def generar_link_tv_pro(ticker):
    """
    Genera enlaces directos al gráfico en TradingView detectando el mercado correcto.
    Prioriza ADRs en NYSE/NASDAQ para mayor volumen, conecta locales si es necesario.
    """
    clean_ticker = ticker.replace('.MX', '').replace('.SA', '')

    # 1. Acciones Locales (.SA = Bovespa, .MX = Bolsa Mexicana)
    if '.SA' in ticker: return f"https://es.tradingview.com/symbols/BMFBOVESPA-{clean_ticker}/"
    if '.MX' in ticker: return f"https://es.tradingview.com/symbols/BMV-{clean_ticker}/"

    # 2. Tech Growth (Generalmente NASDAQ)
    nasdaq_stocks = ['MELI', 'XP', 'STNE', 'PAGS', 'DESP', 'DLO', 'PAC', 'OMAB']
    if clean_ticker in nasdaq_stocks:
        return f"https://es.tradingview.com/symbols/NASDAQ-{clean_ticker}/"

    # 3. Default: NYSE (ADRs Latam Clásicos)
    return f"https://es.tradingview.com/symbols/NYSE-{clean_ticker}/"


def obtener_pais_display(ticker):
    """Etiqueta geográfica limpia para la tarjeta"""
    if '.SA' in ticker or ticker in ['PBR', 'VALE', 'ITUB', 'BBD', 'ABEV', 'NU', 'BSBR', 'GGB', 'SID', 'SUZ', 'SBS']: return 'Brasil'
    if '.MX' in ticker or ticker in ['CX', 'FMX', 'KOF', 'PAC', 'ASR', 'OMAB', 'TV']: return 'México'
    if ticker in ['YPF', 'GGAL', 'BMA', 'PAM', 'TGS', 'VIST', 'TX', 'CAAP']: return 'Argentina'
    if ticker in ['SQM', 'BCH']: return 'Chile'
    if ticker in ['EC', 'CIB']: return 'Colombia'
    if ticker in ['SCCO', 'BAP', 'BVN']: return 'Perú'
    if ticker in ['CPA']: return 'Panamá'
    if ticker in ['MELI', 'GLOB', 'ARCO', 'DLO']: return 'Regional'
    return 'Latam'

# ============================================================================
# 3. LÓGICA DE VISUALIZACIÓN & MOMENTUM
# ============================================================================

def obtener_color_momentum(cambio):
    """
    Escala de colores basada en momentum de precio.
    Rojo/Verde con gradientes de intensidad para identificar fuerza de tendencia.
    """
    if cambio > 5: return '#00a000'    # Verde Intenso (Bullish Fuerte)
    elif cambio > 2: return '#4caf50'  # Verde Medio (Bullish)
    elif cambio > 0: return '#a8d5a8'  # Verde Pálido (Positivo Débil)
    elif cambio > -2: return '#ffb3b3' # Rojo Pálido (Negativo Débil)
    elif cambio > -5: return '#ff6666' # Rojo Medio (Bearish)
    else: return '#cc0000'             # Rojo Intenso (Crash/Corrección)


def obtener_contraste_texto(cambio):
    """Garantiza legibilidad del texto sobre el fondo de color"""
    return 'white' if abs(cambio) > 2 else '#1a1a1a'


def generar_leyenda_visual():
    """Genera la leyenda explicativa del gradiente de momentum"""
    return """
    <div style="margin-top: 30px; padding: 15px; background: white; border-radius: 8px; border: 1px solid #e0e0e0; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="font-size: 0.8em; color: #666; margin-bottom: 10px; font-weight: 700; text-transform: uppercase; letter-spacing: 1px;">
            🌡️ Mapa de Calor - Intensidad de Tendencia (1 Mes)
        </div>
        <div style="display: flex; flex-wrap: wrap; gap: 15px; justify-content: center; font-family: sans-serif;">
            <div style="display:flex; align-items:center; gap:6px;">
                <div style="width:18px; height:18px; background:#00a000; border-radius:4px;"></div>
                <span style="font-size:0.8em; color:#444; font-weight:600;">&gt; +5%</span>
            </div>
            <div style="display:flex; align-items:center; gap:6px;">
                <div style="width:18px; height:18px; background:#4caf50; border-radius:4px;"></div>
                <span style="font-size:0.8em; color:#444;">+2% a +5%</span>
            </div>
            <div style="display:flex; align-items:center; gap:6px;">
                <div style="width:18px; height:18px; background:#a8d5a8; border-radius:4px;"></div>
                <span style="font-size:0.8em; color:#444;">0% a +2%</span>
            </div>
            <div style="display:flex; align-items:center; gap:6px;">
                <div style="width:18px; height:18px; background:#ffb3b3; border-radius:4px;"></div>
                <span style="font-size:0.8em; color:#444;">0% a -2%</span>
            </div>
            <div style="display:flex; align-items:center; gap:6px;">
                <div style="width:18px; height:18px; background:#ff6666; border-radius:4px;"></div>
                <span style="font-size:0.8em; color:#444;">-2% a -5%</span>
            </div>
            <div style="display:flex; align-items:center; gap:6px;">
                <div style="width:18px; height:18px; background:#cc0000; border-radius:4px;"></div>
                <span style="font-size:0.8em; color:#444; font-weight:600;">&lt; -5%</span>
            </div>
        </div>
    </div>
    """

# ============================================================================
# 4. MOTOR DE DATOS
# ============================================================================

def descargar_latam_45():
    """Descarga optimizada de datos financieros con reintentos y manejo robusto"""
    print(f"📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...")
    print(f"📊 Sincronizando LATAM 45 INDEX ({len(LATAM_45_TICKERS)} activos)...")

    datos = {}
    errores_tickers = []

    # Iteración individual para manejo de errores granular
    for idx, (ticker, nombre) in enumerate(LATAM_45_TICKERS.items(), 1):
        intentos_max = 2
        obtenido = False
        
        for intento in range(intentos_max):
            try:
                stock = yf.Ticker(ticker)
                # Solicitamos 1 mes de historia con timeout explícito
                hist = stock.history(period="1mo", timeout=10)

                if not hist.empty and len(hist) > 1:
                    precio_actual = hist['Close'][-1]
                    precio_previo = hist['Close'][0]

                    # Cálculo de Retorno y Volatilidad
                    pct_change = ((precio_actual - precio_previo) / precio_previo) * 100
                    volatilidad = hist['Close'].pct_change().std() * np.sqrt(21) * 100

                    datos[ticker] = {
                        'nombre': nombre,
                        'ticker_display': ticker.replace('.SA','').replace('.MX',''),
                        'cambio_pct': pct_change,
                        'precio': precio_actual,
                        'volatilidad': volatilidad,
                        'pais': obtener_pais_display(ticker),
                        'link': generar_link_tv_pro(ticker)
                    }
                    obtenido = True
                    print(f"  [{idx:2d}/45] ✅ {ticker:15s} → {pct_change:+.2f}%")
                    break
            except Exception as e:
                if intento == intentos_max - 1:
                    errores_tickers.append((ticker, type(e).__name__))
                    print(f"  [{idx:2d}/45] ⚠️  {ticker:15s} → Error: {type(e).__name__}")
                continue

    print(f"\n✅ SINCRONIZACIÓN COMPLETADA: {len(datos)}/{len(LATAM_45_TICKERS)} activos operativos.\n")
    if errores_tickers and len(datos) > 0:
        print(f"   ({len(errores_tickers)} activos con errores de conexión)\n")
    
    return pd.DataFrame.from_dict(datos, orient='index')

# Nota: la implementación de `render_heatmap_grid` se define en una celda separada
# con soporte mejorado de logos y validación. Mantén esa versión y elimina duplicados.


In [4]:
def render_heatmap_grid(df):
    if df.empty: return "<div style='padding:20px; text-align:center'>⏳ Sin datos cargados.</div>"

    css = """
    <style>
        .grid-latam {
            display: grid;
            grid-template-columns: repeat(auto-fill, minmax(210px, 1fr));
            gap: 16px;
            padding: 10px 0;
        }
        .card-link { text-decoration: none; color: inherit; display: block; }
        .card-stock {
            background: white;
            border-radius: 8px;
            padding: 16px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.06);
            transition: transform 0.2s cubic-bezier(0.175, 0.885, 0.32, 1.275), box-shadow 0.2s;
            border: 1px solid #f1f1f1;
        }
        .card-stock:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 20px rgba(0,0,0,0.12);
            z-index: 10;
            border-color: #ddd;
        }
        .card-header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 8px; }
        .badge-ticker {
            background: #2c3e50; color: white; padding: 4px 10px;
            border-radius: 4px; font-size: 0.8em; font-weight: 700; font-family: monospace;
        }
        .tv-indicator { font-size: 0.7em; color: #2962FF; font-weight: 600; text-transform: uppercase; }
        .name-company {
            font-size: 0.9em; color: #555; font-weight: 500;
            margin-bottom: 12px; font-family: 'Segoe UI', sans-serif;
        }
        .box-price {
            text-align: center; padding: 12px; border-radius: 6px;
            font-weight: 800; font-size: 1.5em; margin-bottom: 12px;
            letter-spacing: -0.5px;
        }
        .meta-info { display: flex; justify-content: space-between; font-size: 0.75em; color: #888; border-top: 1px solid #f5f5f5; padding-top: 8px;}
    </style>
    """

    html_cards = ""

    for t, row in df.iterrows():
        pct = row['cambio_pct']
        bg_color = obtener_color_momentum(pct)
        txt_color = obtener_contraste_texto(pct)

        html_cards += f"""
        <a href="{row['link']}" target="_blank" class="card-link" title="Analizar {row['nombre']} en TradingView">
            <div class="card-stock" style="border-top: 4px solid {bg_color};">
                <div class="card-header">
                    <span class="badge-ticker">{row['ticker_display']}</span>
                    <span class="tv-indicator">Chart ↗</span>
                </div>
                <div class="name-company">{row['nombre']}</div>
                <div class="box-price" style="background-color: {bg_color}; color: {txt_color};">
                    {pct:+.2f}%
                </div>
                <div class="meta-info">
                    <span>{row['pais']}</span>
                    <span>${row['precio']:.2f}</span>
                </div>
            </div>
        </a>
        """

    return css + f'<div class="grid-latam">{html_cards}</div>' + generar_leyenda_visual()


In [5]:
# ============================================================================
# DIAGNÓSTICO: Prueba de conexión a yfinance
# ============================================================================

print("🔍 Diagnosticando conexión a yfinance...")

# Prueba 1: Verificar conexión básica
try:
    test_ticker = yf.Ticker("AAPL")
    test_data = test_ticker.history(period="1d", timeout=10)
    if not test_data.empty:
        print("✅ Conexión a yfinance: OK")
    else:
        print("⚠️ yfinance respondió pero sin datos")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__}: {str(e)}")

# Prueba 2: Intentar descargar MELI (activo LATAM más popular)
try:
    meli_test = yf.Ticker("MELI")
    meli_data = meli_test.history(period="1mo", timeout=10)
    print(f"📊 MELI data points: {len(meli_data)}")
except Exception as e:
    print(f"⚠️ Error con MELI: {type(e).__name__}")


🔍 Diagnosticando conexión a yfinance...
✅ Conexión a yfinance: OK
📊 MELI data points: 22


In [6]:
# ============================================================================
# RECARGA CON FUNCIÓN MEJORADA
# ============================================================================

def descargar_latam_45_v2():
    """Descarga optimizada con reintentos y logs detallados"""
    print(f"📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...")
    print(f"📊 Sincronizando LATAM 45 INDEX ({len(LATAM_45_TICKERS)} activos)...\n")

    datos = {}
    errores_tickers = []

    for idx, (ticker, nombre) in enumerate(LATAM_45_TICKERS.items(), 1):
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1mo", timeout=10)

            if not hist.empty and len(hist) > 1:
                precio_actual = hist['Close'][-1]
                precio_previo = hist['Close'][0]
                pct_change = ((precio_actual - precio_previo) / precio_previo) * 100
                volatilidad = hist['Close'].pct_change().std() * np.sqrt(21) * 100

                datos[ticker] = {
                    'nombre': nombre,
                    'ticker_display': ticker.replace('.SA','').replace('.MX',''),
                    'cambio_pct': pct_change,
                    'precio': precio_actual,
                    'volatilidad': volatilidad,
                    'pais': obtener_pais_display(ticker),
                    'link': generar_link_tv_pro(ticker)
                }
                print(f"  [{idx:2d}/45] ✅ {ticker:15s} → {pct_change:+.2f}%")
        except Exception as e:
            errores_tickers.append(ticker)
            print(f"  [{idx:2d}/45] ⚠️  {ticker:15s} → {type(e).__name__}")

    print(f"\n✅ SINCRONIZACIÓN COMPLETADA: {len(datos)}/{len(LATAM_45_TICKERS)} activos operativos.\n")
    return pd.DataFrame.from_dict(datos, orient='index')

# Ejecutar descarga mejorada
df_mercado_v2 = descargar_latam_45_v2()

# Mostrar estadísticas
if not df_mercado_v2.empty:
    print(f"\n📊 RESUMEN DE DATOS:")
    print(f"   • Cambio promedio: {df_mercado_v2['cambio_pct'].mean():+.2f}%")
    print(f"   • Alcistas: {(df_mercado_v2['cambio_pct'] > 0).sum()}")
    print(f"   • Bajistas: {(df_mercado_v2['cambio_pct'] < 0).sum()}")
    print(f"   • Precio promedio: ${df_mercado_v2['precio'].mean():.2f}")


📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...
📊 Sincronizando LATAM 45 INDEX (45 activos)...

  [ 1/45] ⚠️  MELI            → KeyError
  [ 2/45] ⚠️  NU              → KeyError
  [ 3/45] ⚠️  GLOB            → KeyError
  [ 4/45] ⚠️  STNE            → KeyError
  [ 5/45] ⚠️  PAGS            → KeyError
  [ 6/45] ⚠️  XP              → KeyError
  [ 7/45] ⚠️  DLO             → KeyError
  [ 8/45] ⚠️  PBR             → KeyError
  [ 9/45] ⚠️  YPF             → KeyError
  [10/45] ⚠️  VIST            → KeyError
  [11/45] ⚠️  PAM             → KeyError
  [12/45] ⚠️  TGS             → KeyError
  [13/45] ⚠️  CMIG4.SA        → KeyError
  [14/45] ⚠️  CPFE3.SA        → KeyError
  [15/45] ⚠️  UGPA3.SA        → KeyError
  [16/45] ⚠️  SBS             → KeyError
  [17/45] ⚠️  VALE            → KeyError
  [18/45] ⚠️  SCCO            → KeyError
  [19/45] ⚠️  SQM             → KeyError
  [20/45] ⚠️  TX              → KeyError
  [21/45] ⚠️  GGB             → KeyError
  [22/45] ⚠️  SID             → KeyErr

In [7]:
# ============================================================================
# DIAGNÓSTICO: Investigar estructura de datos yfinance
# ============================================================================

print("🔍 Investigando estructura de datos de yfinance...\n")

# Descargar datos de prueba
ticker_test = yf.Ticker("MELI")
hist_test = ticker_test.history(period="1mo", timeout=10)

print(f"Tipo de datos: {type(hist_test)}")
print(f"Forma: {hist_test.shape}")
print(f"Columnas disponibles: {list(hist_test.columns)}")
print(f"\nPrimas 3 filas:")
print(hist_test.head(3))
print(f"\nÚltimas 3 filas:")
print(hist_test.tail(3))
print(f"\nÍndice:")
print(hist_test.index)
print(f"\nTipos de datos:")
print(hist_test.dtypes)


🔍 Investigando estructura de datos de yfinance...

Tipo de datos: <class 'pandas.DataFrame'>
Forma: (22, 7)
Columnas disponibles: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']

Primas 3 filas:
                                  Open         High         Low        Close  \
Date                                                                           
2026-01-07 00:00:00-05:00  2187.040039  2188.489990  2130.00000  2162.610107   
2026-01-08 00:00:00-05:00  2176.699951  2200.699951  2160.02002  2179.800049   
2026-01-09 00:00:00-05:00  2191.500000  2193.060059  2162.00000  2178.409912   

                           Volume  Dividends  Stock Splits  
Date                                                        
2026-01-07 00:00:00-05:00  383100        0.0           0.0  
2026-01-08 00:00:00-05:00  340500        0.0           0.0  
2026-01-09 00:00:00-05:00  309200        0.0           0.0  

Últimas 3 filas:
                                  Open         High      

In [8]:
# Verificar si 'Close' existe
print("¿Existe columna 'Close'?", 'Close' in hist_test.columns)
print("¿Existe columna 'close'?", 'close' in hist_test.columns)

# Intentar acceder de forma segura
try:
    valor = hist_test['Close'].iloc[-1]
    print(f"✅ Acceso exitoso a Close: {valor}")
except KeyError as e:
    print(f"❌ KeyError al acceder a Close: {e}")
    print(f"   Columnas disponibles: {list(hist_test.columns)}")
    
    # Intentar con la primera columna de precio disponible
    if len(hist_test.columns) > 0:
        col_precio = hist_test.columns[0]
        print(f"   Usando columna alternativa: {col_precio}")
        print(f"   Valor: {hist_test[col_precio].iloc[-1]}")


¿Existe columna 'Close'? True
¿Existe columna 'close'? False
✅ Acceso exitoso a Close: 1970.1500244140625


In [9]:
# ============================================================================
# RECARGA CON MEJOR MANEJO DE ERRORES
# ============================================================================

def descargar_latam_45_v3():
    """Descarga con manejo robusto de errores y diagnóstico"""
    print(f"📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...")
    print(f"📊 Sincronizando LATAM 45 INDEX ({len(LATAM_45_TICKERS)} activos)...\n")

    datos = {}
    errores_detalles = []

    for idx, (ticker, nombre) in enumerate(LATAM_45_TICKERS.items(), 1):
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1mo", timeout=10)

            # Validación: verificar que histórico tenga datos y 'Close'
            if hist.empty:
                raise ValueError("Histórico vacío")
            
            if 'Close' not in hist.columns:
                raise ValueError(f"Columna 'Close' no encontrada. Disponibles: {list(hist.columns)}")
            
            if len(hist) < 2:
                raise ValueError(f"Histórico tiene {len(hist)} filas, se necesitan al menos 2")

            # Extraer precios de forma segura
            precio_actual = hist['Close'].iloc[-1]
            precio_previo = hist['Close'].iloc[0]

            # Evitar división por cero
            if precio_previo == 0:
                raise ValueError("Precio previo es cero")

            pct_change = ((precio_actual - precio_previo) / precio_previo) * 100
            volatilidad = hist['Close'].pct_change().std() * np.sqrt(21) * 100

            datos[ticker] = {
                'nombre': nombre,
                'ticker_display': ticker.replace('.SA','').replace('.MX',''),
                'cambio_pct': pct_change,
                'precio': precio_actual,
                'volatilidad': volatilidad,
                'pais': obtener_pais_display(ticker),
                'link': generar_link_tv_pro(ticker)
            }
            print(f"  [{idx:2d}/45] ✅ {ticker:15s} → {pct_change:+.2f}%")

        except Exception as e:
            error_msg = f"{type(e).__name__}: {str(e)[:50]}"
            errores_detalles.append((ticker, error_msg))
            print(f"  [{idx:2d}/45] ⚠️  {ticker:15s} → {type(e).__name__}")

    print(f"\n✅ SINCRONIZACIÓN COMPLETADA: {len(datos)}/{len(LATAM_45_TICKERS)} activos operativos.\n")
    
    if errores_detalles:
        print(f"⚠️  Errores detectados ({len(errores_detalles)}):")
        for ticker, error in errores_detalles[:5]:  # Mostrar primeros 5
            print(f"   • {ticker}: {error}")
        if len(errores_detalles) > 5:
            print(f"   ... y {len(errores_detalles) - 5} más")
    
    return pd.DataFrame.from_dict(datos, orient='index')

# Ejecutar con diagnóstico mejorado
print("Ejecutando descargar_latam_45_v3()...\n")
df_mercado_v3 = descargar_latam_45_v3()
print(f"\n📊 DataFrame resultante: {df_mercado_v3.shape[0]} activos cargados")


Ejecutando descargar_latam_45_v3()...

📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...
📊 Sincronizando LATAM 45 INDEX (45 activos)...

  [ 1/45] ✅ MELI            → -8.90%
  [ 2/45] ✅ NU              → -0.57%
  [ 3/45] ✅ GLOB            → -14.68%
  [ 4/45] ✅ STNE            → +21.39%
  [ 5/45] ✅ PAGS            → +14.24%
  [ 6/45] ✅ XP              → +15.14%
  [ 7/45] ✅ DLO             → -12.02%
  [ 8/45] ✅ PBR             → +28.86%
  [ 9/45] ✅ YPF             → +16.35%
  [10/45] ✅ VIST            → +27.48%
  [11/45] ✅ PAM             → +2.43%
  [12/45] ✅ TGS             → +1.41%
  [13/45] ✅ CMIG4.SA        → +2.07%
  [14/45] ✅ CPFE3.SA        → -7.49%
  [15/45] ✅ UGPA3.SA        → +26.52%
  [16/45] ✅ SBS             → +15.01%
  [17/45] ✅ VALE            → +14.31%
  [18/45] ✅ SCCO            → +26.54%
  [19/45] ✅ SQM             → -4.82%
  [20/45] ✅ TX              → +6.86%
  [21/45] ✅ GGB             → +6.03%
  [22/45] ✅ SID             → -2.66%
  [23/45] ✅ CX              → +5.8

In [10]:
# Re-definición de descargar_latam_45_v3 usando fast_info cuando esté disponible

def descargar_latam_45_v3():
    """Descarga con manejo robusto de errores y uso de fast_info para precio más actual"""
    print(f"📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...")
    print(f"📊 Sincronizando LATAM 45 INDEX ({len(LATAM_45_TICKERS)} activos)...\n")

    datos = {}
    errores_detalles = []

    for idx, (ticker, nombre) in enumerate(LATAM_45_TICKERS.items(), 1):
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1mo", timeout=10)

            # Validación: verificar que histórico tenga datos y 'Close'
            if hist.empty:
                raise ValueError("Histórico vacío")

            if 'Close' not in hist.columns:
                raise ValueError(f"Columna 'Close' no encontrada. Disponibles: {list(hist.columns)}")

            if len(hist) < 2:
                raise ValueError(f"Histórico tiene {len(hist)} filas, se necesitan al menos 2")

            # Precio actual con fast_info fallback
            precio_actual = stock.fast_info.get('lastPrice', hist['Close'].iloc[-1])
            precio_previo = hist['Close'].iloc[0]

            # Evitar división por cero
            if precio_previo == 0:
                raise ValueError("Precio previo es cero")

            pct_change = ((precio_actual - precio_previo) / precio_previo) * 100
            volatilidad = hist['Close'].pct_change().std() * np.sqrt(21) * 100

            datos[ticker] = {
                'nombre': nombre,
                'ticker_display': ticker.replace('.SA','').replace('.MX',''),
                'cambio_pct': pct_change,
                'precio': precio_actual,
                'volatilidad': volatilidad,
                'pais': obtener_pais_display(ticker),
                'link': generar_link_tv_pro(ticker)
            }
            print(f"  [{idx:2d}/45] ✅ {ticker:15s} → {pct_change:+.2f}%")

        except Exception as e:
            error_msg = f"{type(e).__name__}: {str(e)[:50]}"
            errores_detalles.append((ticker, error_msg))
            print(f"  [{idx:2d}/45] ⚠️  {ticker:15s} → {type(e).__name__}")

    print(f"\n✅ SINCRONIZACIÓN COMPLETADA: {len(datos)}/{len(LATAM_45_TICKERS)} activos operativos.\n")

    if errores_detalles:
        print(f"⚠️  Errores detectados ({len(errores_detalles)}):")
        for ticker, error in errores_detalles[:5]:  # Mostrar primeros 5
            print(f"   • {ticker}: {error}")
        if len(errores_detalles) > 5:
            print(f"   ... y {len(errores_detalles) - 5} más")

    return pd.DataFrame.from_dict(datos, orient='index')

# Ejecutar con la versión actualizada
df_mercado_v3 = descargar_latam_45_v3()
print(f"\n📊 DataFrame resultante: {df_mercado_v3.shape[0]} activos cargados")

📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...
📊 Sincronizando LATAM 45 INDEX (45 activos)...

  [ 1/45] ✅ MELI            → -8.90%
  [ 2/45] ✅ NU              → -0.57%
  [ 3/45] ✅ GLOB            → -14.68%
  [ 4/45] ✅ STNE            → +21.39%
  [ 5/45] ✅ PAGS            → +14.24%
  [ 6/45] ✅ XP              → +15.14%
  [ 7/45] ✅ DLO             → -12.02%
  [ 8/45] ✅ PBR             → +28.86%
  [ 9/45] ✅ YPF             → +16.35%
  [10/45] ✅ VIST            → +27.48%
  [11/45] ✅ PAM             → +2.43%
  [12/45] ✅ TGS             → +1.41%
  [13/45] ✅ CMIG4.SA        → +2.07%
  [14/45] ✅ CPFE3.SA        → -7.49%
  [15/45] ✅ UGPA3.SA        → +26.52%
  [16/45] ✅ SBS             → +15.01%
  [17/45] ✅ VALE            → +14.31%
  [18/45] ✅ SCCO            → +26.54%
  [19/45] ✅ SQM             → -4.82%
  [20/45] ✅ TX              → +6.86%
  [21/45] ✅ GGB             → +6.03%
  [22/45] ✅ SID             → -2.66%
  [23/45] ✅ CX              → +5.81%
  [24/45] ✅ SUZ             → +3.97%

In [11]:
# ============================================================================
# AUTO-REFRESH
# ============================================================================

def auto_refresh(interval=60):
    global df_mercado_v3

    while True:
        try:
            df_mercado_v3 = descargar_latam_45_v3()
            clear_output(wait=True)
            lanzar_terminal_latam_45(df_mercado_v3)
        except Exception as e:
            print("⚠️ Error en auto-refresh:", e)

        time.sleep(interval)


In [12]:
# Mostrar resumen del resultado
print("=" * 60)
print("RESUMEN DE EJECUCIÓN")
print("=" * 60)
print(f"Activos cargados exitosamente: {len(df_mercado_v3)}")

if len(df_mercado_v3) > 0:
    print(f"\nPrimeros 5 activos:")
    print(df_mercado_v3[['nombre', 'cambio_pct', 'precio']].head())
    print(f"\nEstadísticas:")
    print(f"  Cambio promedio: {df_mercado_v3['cambio_pct'].mean():+.2f}%")
    print(f"  Alcistas: {(df_mercado_v3['cambio_pct'] > 0).sum()}")
    print(f"  Bajistas: {(df_mercado_v3['cambio_pct'] < 0).sum()}")
else:
    print("❌ No se cargaron activos. El dashboard no podrá mostrar datos.")


RESUMEN DE EJECUCIÓN
Activos cargados exitosamente: 45

Primeros 5 activos:
             nombre  cambio_pct       precio
MELI   MercadoLibre   -8.899435  1970.150024
NU    Nubank Global   -0.571431    17.400000
GLOB        Globant  -14.681640    60.029999
STNE        StoneCo   21.393037    17.080000
PAGS      PagSeguro   14.241163    10.990000

Estadísticas:
  Cambio promedio: +9.27%
  Alcistas: 37
  Bajistas: 8


In [13]:
# ============================================================================
# LANZAR DASHBOARD CON AUTO-ACTUALIZACIÓN DE PRECIOS
# ============================================================================

print("\n" + "="*60)
print("🎯 INICIANDO DASHBOARD INTERACTIVO LATAM 45 INDEX")
print("="*60 + "\n")

# Cargar datos iniciales
df_mercado_v3 = descargar_latam_45_v3()

# Widget para mostrar el dashboard
output_dashboard = widgets.Output()

def actualizar_dashboard():
    """Actualiza continuamente el dashboard con precios nuevos"""
    global df_mercado_v3
    
    with output_dashboard:
        while True:
            try:
                # Descargar datos nuevos
                df_mercado_v3 = descargar_latam_45_v3()
                
                # Limpiar y re-renderizar
                clear_output(wait=True)
                lanzar_terminal_latam_45(df_mercado_v3)
                
                print(f"\n⏰ Próxima actualización en 60 segundos...")
                
            except Exception as e:
                print(f"⚠️ Error en actualización: {e}")
            
            time.sleep(60)

# Mostrar dashboard inicial
lanzar_terminal_latam_45(df_mercado_v3)
display(output_dashboard)

# Botón para iniciar auto-refresh
btn_refresh = widgets.Button(
    description="▶️ Iniciar Auto-Actualización",
    button_style="success",
    tooltip="Actualiza precios cada 60 segundos"
)

btn_stop = widgets.Button(
    description="⏹️ Detener",
    button_style="danger",
    tooltip="Detiene la actualización"
)

hilo_refresh = None

def iniciar_refresh(b):
    global hilo_refresh
    if hilo_refresh is None or not hilo_refresh.is_alive():
        hilo_refresh = threading.Thread(
            target=actualizar_dashboard,
            daemon=True
        )
        hilo_refresh.start()
        print("🟢 Auto-actualización iniciada (60s)")
    else:
        print("⚠️ Auto-actualización ya está activa")

def detener_refresh(b):
    global hilo_refresh
    if hilo_refresh and hilo_refresh.is_alive():
        print("🔴 Auto-actualización detenida")
    else:
        print("⚠️ Auto-actualización no estaba activa")

btn_refresh.on_click(iniciar_refresh)
btn_stop.on_click(detener_refresh)

controles = widgets.HBox([btn_refresh, btn_stop])
display(controles)



🎯 INICIANDO DASHBOARD INTERACTIVO LATAM 45 INDEX

📡 INICIANDO CONEXIÓN SEGURA CON MERCADOS GLOBALES...
📊 Sincronizando LATAM 45 INDEX (45 activos)...

  [ 1/45] ✅ MELI            → -8.90%
  [ 2/45] ✅ NU              → -0.57%
  [ 3/45] ✅ GLOB            → -14.68%
  [ 4/45] ✅ STNE            → +21.39%
  [ 5/45] ✅ PAGS            → +14.24%
  [ 6/45] ✅ XP              → +15.14%
  [ 7/45] ✅ DLO             → -12.02%
  [ 8/45] ✅ PBR             → +28.86%
  [ 9/45] ✅ YPF             → +16.35%
  [10/45] ✅ VIST            → +27.48%
  [11/45] ✅ PAM             → +2.43%
  [12/45] ✅ TGS             → +1.41%
  [13/45] ✅ CMIG4.SA        → +2.07%
  [14/45] ✅ CPFE3.SA        → -7.49%
  [15/45] ✅ UGPA3.SA        → +26.52%
  [16/45] ✅ SBS             → +15.01%
  [17/45] ✅ VALE            → +14.31%
  [18/45] ✅ SCCO            → +26.54%
  [19/45] ✅ SQM             → -4.82%
  [20/45] ✅ TX              → +6.86%
  [21/45] ✅ GGB             → +6.03%
  [22/45] ✅ SID             → -2.66%
  [23/45] ✅ CX        

NameError: name 'lanzar_terminal_latam_45' is not defined